In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%pwd
%cd '/content/drive/MyDrive/langchain'
%pwd

/content/drive/MyDrive/langchain


'/content/drive/MyDrive/langchain'

In [ ]:
!pip -q install langchain
!pip install openai
!pip -q install chromadb
!pip -q install --upgrade together
!pip -q install -U FlagEmbedding
!pip install unstructured
!pip install cohere
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all th

In [ ]:
import os
from together_api_key import together_api_key

from cohere_api_key import cohere_api_key
import cohere

together_model_name = "togethercomputer/llama-2-70b-chat"
os.environ["TOGETHER_API_KEY"] = together_api_key  # your together api key
#m="togethercomputer/llama-2-70b-chat"
# togethercomputer/llama-2-13b-chat

os.environ["COHERE_API_KEY"]=cohere_api_key
os.environ["OPENAI_API_KEY"] = "" # your open ai key

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import langchain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.schema.output_parser import StrOutputParser

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

# hugging face pipe line처럼 together pipe line 만들기

class TogetherLLM(LLM):

    # set model
    model:str = together_model_name

    # set api key
    together_api_key:str = os.environ["TOGETHER_API_KEY"]

    # set temperature
    temperature:float = 0.5

    max_tokens:int = 1024


    class Config:
        extra = Extra.forbid

    # #@root_validator(skip_on_failure=True)
    #
    #
    # def validate_environment(cls,values: Dict)->Dict:

    #     # 등록한 together api를 validate한다
    #     api_key = get_from_dict_or_env(
    #         values,"together_api_key","TOGETHER_API_KEY"
    #     )
    #     values["together_api_key"] = api_key
    #     return values

    # llm type is together
    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    # llm run 할때 call
    def _call(self,prompt,**kwargs) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text

<ipython-input-6-22407ff89e24>:30: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  extra = Extra.forbid


In [ ]:
llm = TogetherLLM(
    model= together_model_name,
    temperature=0.0,
    max_tokens=1024

)

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

#라마2맞게 prompt조합

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

In [ ]:
from typing import List, Optional


from pydantic import BaseModel, Field, validator


from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_extraction_chain_pydantic
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from pathlib import Path
from langchain.output_parsers import PydanticOutputParser
from langchain import PromptTemplate,LLMChain
from typing import Sequence
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.document_loaders import DirectoryLoader

In [ ]:
# To Extract schema info


Personality_schema = ResponseSchema(name="Personality",
                                   description="Personality is a term used to describe the unique patterns of thoughts, feelings, and behaviors that distinguish one person from another"
                                   )

Value_schema = ResponseSchema(name="Values",
                              description="Values are the guiding principles that shape our lives and influence our behavior. They are the beliefs and ideals that we hold dear, and they help us make decisions about what is important to us "
                              )

Expertise_schema = ResponseSchema(name="Expertise",
            description="Expertise is a term used to describe a high level of knowledge or skill in a particular field"
            )

Experience_schema = ResponseSchema(name="Experience",
                                   description="Experience is a term used to describe the process of gaining knowledge or skill from doing, seeing, or feeling things"
                                   )
Interests_schema = ResponseSchema(name="Interests",
                                  description="Interest is a term used to describe the feeling of wanting to give your attention to something or of wanting to be involved with and to discover more about something"
                                  )
response_schemas = [Personality_schema,
                    Value_schema,
                    Expertise_schema,
                    Experience_schema,
                    Interests_schema
                    ]

In [ ]:
schema_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
schema_format_instructions = schema_output_parser.get_format_instructions()

In [ ]:
instruction = """
Extract the information about Personality, Values, Expertise, Experience, and Interests from the provided text. If any information cannot be extracted, leave it blank.

Text: {text}

Format Instructions: {format_instructions}

each keys must Finish with one sentence. 1 line, 1 column.

most important thing is Follow JSON FORMAT!

Format the output as JSON with the following keys :

"""


schema_template=get_prompt(instruction)

schema_prompt = PromptTemplate(template=schema_template,input_variables=["text"],
                        partial_variables={"format_instructions": schema_format_instructions})




In [ ]:
mentor_mentee_name=[]

In [ ]:
# 유료 파파고 api

client_id = "" # 개발자센터에서 발급받은 Client ID 값
client_secret = "" # 개발자센터에서 발급받은 Client Secret 값
url = ""



In [ ]:
# mentor info

loader_mentee = DirectoryLoader("./data/langchain_data/",glob="mentor_info.txt")
data_mentor=loader_mentee.load()

mentor_markdown_doc=data_mentor[0].page_content

headers_to_split_on = [
    ("#", "Name"),
]

mentor_dict={}
mentor_doc_dict={}
mentor_attribute_dic={}


mentor_markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
mentor_md_header_splits = mentor_markdown_splitter.split_text(mentor_markdown_doc)

mentor_doc=mentor_md_header_splits

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
mentor_doc

[Document(page_content='안녕하세요, 저는 책임감과 데이터 중심의 의사 결정을 선호하는 기업 재무 분석가 Trex입니다.저의 핵심 가치는 기업의 재무 건전성을 면밀히 조사하고 미래 성장 가능성을 평가하는 것에 있습니다.글로벌 기업 재무 분석가로서 12년 이상의 경험을 쌓으며 기업 재무 분석과 재무 모델링에 대한 깊은 이해를 쌓았습니다.저의 주된 관심사는 기업의 재무 건전성을 보장하고 효과적인 투자 전략을 세우는 것입니다.저는 제 전문 지식을 공유하고 여러분이 기업 금융의 세계를 개척하는 것을 돕기 위해 여기에 왔습니다.', metadata={'Name': 'Trex'}),
 Document(page_content='안녕하세요! 여러분들의 금융 여정에 함께하게 된 멘토로서 매우 기쁩니다.나는 금융 분야에서의 전문 지식을 바탕으로 재테크(financial technology)와 투자에 대한 실질적이고 현실적인 조언을 제공하는 것을 즐깁니다. 금융 전략의 핵심과 효율적인 자산 관리에 대한 이해를 통해 여러분의 재무적인 목표를 달성하는 데 도움이 되고 싶어졌어요.내 성격은 적극적이고 실용적인 편입니다. 무엇보다도, 여러분의 목표 달성을 위해 힘쓰는 모습을 지켜보는 것이 가장 큰 보람입니다. 금융의 복잡성을 단순하게 전달하고, 실제로 적용 가능한 솔루션을 제시하는 것을 목표로 하고 있습니다.가치관 측면에서는 돈을 통한 가치 창출을 중요하게 여기며, 금융 지식을 활용하여 개인과 사회적인 성장에 기여하고자 합니다. 함께 일하면서 여러분의 금융적인 목표를 함께 성취하고, 돈이 주는 자유로움을 함께 누려 나가길 기대합니다. 함께 여행하는 동안 서로에게 영감을 주며, 긍정적이고 풍요로운 여정을 만들어 나가요!', metadata={'Name': 'Sulley'}),
 Document(page_content='안녕하세요, 저는 Ken입니다.저는 항상 재정적인 문제에 대해 차분하고 분석적인 접근을 해왔습니다.지난 15년 동안 저는 투자 은행 업무와 자산 관리 업무

In [ ]:
import os
import sys
import ast
import urllib.request
import time

for i in range(int(len(mentor_md_header_splits))):

    encText = urllib.parse.quote(mentor_md_header_splits[i].page_content)
    data = "source=ko&target=en&text=" + encText

    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()

        #mentor_markdown_doc[mentor_md_header_splits[i].metadata["Name"]]=ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]

        mentor_mentee_name.append(mentor_md_header_splits[i].metadata["Name"])
        mentor_doc_dict[mentor_md_header_splits[i].metadata["Name"]] = ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]

        _input = schema_prompt.format_prompt(text=ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"])

        ot=llm(_input.to_string())
        print("DEBUG : ",ot)
        time.sleep(0.5)
        try:
            mentor_attribute_dic[mentor_md_header_splits[i].metadata["Name"]] = schema_output_parser.parse(ot)
            time.sleep(0.5)
        except Exception:
            print("Exception")
            time.sleep(0.5)
            mentor_attribute_dic[mentor_md_header_splits[i].metadata["Name"]] = schema_output_parser.parse(ot)
            time.sleep(0.5)


    else:
        print("Error Code:" + rescode)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


DEBUG :   ```json
{
  "Personality": "Responsible and data-driven",
  "Values": "Scrutinizing financial health and evaluating future growth potential",
  "Expertise": "Corporate financial analysis and financial modeling",
  "Experience": "12 years as a global corporate financial analyst",
  "Interests": "Sharing expertise and helping others pioneer the world of corporate finance"
}
```
DEBUG :   ```json
{
  "Personality": "active and practical",
  "Values": "creation of value through money, personal and social growth",
  "Expertise": "finance, financial technology, investment",
  "Experience": "mentorship, financial advice",
  "Interests": "helping others achieve financial goals, contributing to personal and social growth"
}
```
DEBUG :   ```json
{
  "Personality": "Calm and analytical",
  "Values": "Financial stability and planning",
  "Expertise": "Investment banking and asset management",
  "Experience": "15 years in investment banking and asset management",
  "Interests": "Helping 

In [ ]:
mentor_doc_dict

{'Trex': "Hi, I'm Trex, a corporate financial analyst who prefers responsibility and data-driven decision-making.My core value lies in scrutinizing the financial health of a company and evaluating its future growth potential.With more than 12 years of experience as a global corporate financial analyst, he has gained a deep understanding of corporate financial analysis and financial modeling.My main concern is to ensure the financial health of the company and to establish effective investment strategies.I'm here to share my expertise and help you pioneer the world of corporate finance.",
 'Sulley': "Hello! I'm very happy as a mentor to join you on your financial journey.I enjoy providing practical and realistic advice on financial technology and investment based on my expertise in finance. I wanted to help you achieve your financial goals through an understanding of the core of your financial strategy and efficient asset management.My personality is active and practical. Above all, it i

In [ ]:
mentor_attribute_dic

{'Trex': {'Personality': 'Responsible and data-driven',
  'Values': 'Scrutinizing financial health and evaluating future growth potential',
  'Expertise': 'Corporate financial analysis and financial modeling',
  'Experience': '12 years as a global corporate financial analyst',
  'Interests': 'Sharing expertise and helping others pioneer the world of corporate finance'},
 'Sulley': {'Personality': 'active and practical',
  'Values': 'creation of value through money, personal and social growth',
  'Expertise': 'finance, financial technology, investment',
  'Experience': 'mentorship, financial advice',
  'Interests': 'helping others achieve financial goals, contributing to personal and social growth'},
 'Ken': {'Personality': 'Calm and analytical',
  'Values': 'Financial stability and planning',
  'Expertise': 'Investment banking and asset management',
  'Experience': '15 years in investment banking and asset management',
  'Interests': 'Helping people diversify their financial portfolios

In [ ]:
#mentee info

loader_mentee = DirectoryLoader("./data/langchain_data/",glob="mentee_info.txt")
data_mentee=loader_mentee.load()

mentee_markdown_doc=data_mentee[0].page_content

headers_to_split_on = [
    ("#", "Name"),

]
mentee_dict={}
mentee_doc_dict={}
mentee_attr_dic={}

mentee_markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
mentee_md_header_splits = mentee_markdown_splitter.split_text(mentee_markdown_doc)

In [ ]:
# 한 -> 영

for i in range(int(len(mentee_md_header_splits))):

    encText = urllib.parse.quote(mentee_md_header_splits[i].page_content)
    data = "source=ko&target=en&text=" + encText

    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        #mentee_markdown_doc[mentee_md_header_splits[i].metadata["Name"]]=ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]

        mentor_mentee_name.append(mentee_md_header_splits[i].metadata["Name"])
        mentee_doc_dict[mentee_md_header_splits[i].metadata["Name"]] = ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]

        _input = schema_prompt.format_prompt(text=ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"])

        ot=llm(_input.to_string())
        print("DEBUG : ",ot)
        time.sleep(0.5)
        try:
            mentee_attr_dic[mentee_md_header_splits[i].metadata["Name"]]=schema_output_parser.parse(ot)
            time.sleep(0.5)

        except Exception:
            mentee_attr_dic[mentee_md_header_splits[i].metadata["Name"]]=schema_output_parser.parse(ot)
            time.sleep(0.5)
    else:
        print("Error Code:" + rescode)

DEBUG :   ```json
{
  "Personality": "Full of curiosity and passion",
  "Values": "Financial stability through education and contemplation of future investment",
  "Expertise": "Acquiring new knowledge and growing through various experiences in the financial world",
  "Experience": "Building a solid financial foundation, steadily developing savings habits",
  "Interests": "Investment, financial stability, education, and personal growth"
}
```
DEBUG :   ```json
{
  "Personality": "focused on the joy of knowledge and strengthening professionalism",
  "Values": "discovering my own career path through exploration in college and finance",
  "Expertise": "continuously trying to understand various aspects of finance",
  "Experience": "exploration of various flows and trends in the stock market",
  "Interests": "innovate our existing investment strategies in a modern and efficient way"
}
```
DEBUG :   ```json
{
  "Personality": "Strong desire for learning and passion for career development, de

In [ ]:

loader = DirectoryLoader('./data/langchain_data/', glob="./*.pdf", loader_cls=PyPDFLoader)

#PyPDFLoader
documents = loader.load()

#splitting the text into
pdf_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = pdf_text_splitter.split_documents(documents)

len(texts)

38

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
#cromdb 생성 및 임베딩 사용
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = model_norm

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 4}) # default


In [ ]:
# Recommend sys

# set prompt
sys_prompt = """You are a helpful, mentor mentee matching recommender assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """
Answer the question based on following information.

Question:/n/n {question}/n
Please provide the reasonable matching result between the mentor and mentee based on their Python dictionary attributes: Personality, Values, Expertise, Experience, and Interests

Task is Recommend system about mentor, mentee information.

Please provide a reasonable matching result between the mentor and mentee.
Not have any text after reason for the answer is done.

Question: Provide a reasonable matching result between the mentor and mentee based on their each respective attributes, given the following each information :

Mentor Personality: {mentor_p},
Mentor Values: {mentor_v},
Mentor Expertise: {mentor_e},
Mentor Experience: {mentor_ee},
Mentor Interests: {mentor_i},


Mentee Personality: {mentee_p},
Mentee Values : {mentee_v},
Mentee Expertise: {mentee_e},
Mentee Experience: {mentee_ee},
Mentee Interests: {mentee_i},

Answer Format: Reason: Positive Opinion: Negative Opinion:

Please provide the positive and negative opinions regarding the reasonable matching result of the mentor and mentee.
Please don't share false information.
and MUST IMPORTANT is answer in the following Answer Format Resone : Positive Opinion: Negative Opinion: ".
"""

In [ ]:
llm = TogetherLLM(
    model= together_model_name,
    temperature=0.0,
    max_tokens=1024
)

In [ ]:
from langchain.schema import prompt

prompt_template = get_prompt(instruction, sys_prompt)


prompt = ChatPromptTemplate.from_template(prompt_template) # or template

chain = {
    "question": itemgetter("question") | retriever,
    "mentor_p": itemgetter("mentor_p"),
    "mentor_v": itemgetter("mentor_v"),
    "mentor_e": itemgetter("mentor_e"),
    "mentor_ee": itemgetter("mentor_ee"),
    "mentor_i": itemgetter("mentor_i"),
    "mentee_p": itemgetter("mentee_p"),
    "mentee_v": itemgetter("mentee_v"),
    "mentee_e": itemgetter("mentee_e"),
    "mentee_ee": itemgetter("mentee_ee"),
    "mentee_i": itemgetter("mentee_i"),
} | prompt | llm | StrOutputParser()

In [ ]:
import time
import os
import sys
import urllib.request
import ast
import re


retrival_mentee_obj="Aiden" # 검색할 대상 (멘티로 가정)
obj_dic={} # obj에 대한 딕셔너리 각 멘토들과 obj와의 매칭 결과가 저장 된다


for k,v in mentor_attribute_dic.items():
    time.sleep(0.001)
    for mk,mv in mentee_attr_dic.items():

        if(mk==retrival_mentee_obj):

            print(k," ",mk)
            print()

            temp_text=chain.invoke({"question":"Is mentor and mentee matching valid? Tell me the reason, too",
                                    "mentor_p": v["Personality"],
                                    "mentor_v": v["Values"],
                                    "mentor_e": v["Expertise"],
                                    "mentor_ee":v["Experience"],
                                    "mentor_i": v["Interests"],

                                    "mentee_p": mv["Personality"],
                                    "mentee_v": mv["Values"],
                                    "mentee_e": mv["Expertise"],
                                    "mentee_ee": mv["Experience"],
                                    "mentee_i": mv["Interests"],
                                    })
            obj_dic[k]=temp_text
            # match_ = re.search(r'Reason:(.*)', temp_text, re.DOTALL)
            # if match_:
            #     reason_content = match_.group(1).strip()

            #     obj_dic[k]=reason_content
            # else:
            #     print("NOT STORE IN DIC : <PRINT> obj_dic dose not store")

            korText = urllib.parse.quote(obj_dic[k])
            time.sleep(0.001)

            data = "source=en&target=ko&text=" + korText

            request = urllib.request.Request(url)
            request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
            request.add_header("X-NCP-APIGW-API-KEY",client_secret)
            response = urllib.request.urlopen(request, data=data.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                print(ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]) #ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]
                time.sleep(0.001)
            else:
                print("Error Code:" + rescode)

            time.sleep(0.001)
            print()

Trex   Aiden

이유: 멘토와 멘티는 비슷한 가치관, 전문성, 관심사를 공유하며 멘토는 해당 분야에 대한 경험이 풍부하다.

긍정적인 의견: 멘토의 책임감 있고 데이터 중심적인 성격은 멘티의 학습욕구와 단호한 성격을 보완할 것이며, 그들이 공유하는 금융 이해력과 효율성의 가치는 멘티의 멘토링 관계에 강력한 기반을 만들어 줄 것이다. 기업의 재무 분석과 재무 모델링에 대한 멘토의 전문성은 멘티의 재무 기술, 투자 및 재무 전략에 대한 관심에 귀중한 통찰력을 제공할 것이다. 멘토의 12년 글로벌 기업 재무분석가로서의 경험은 멘티가 배울 수 있는 실질적인 지식과 경험을 제공할 것이다.

부정적인 의견: 없음.

Sulley   Aiden

이유: 멘토와 멘티는 유사한 가치관, 전문성, 관심사를 공유하고 멘토는 멘티의 목표와 일치하는 멘토링 및 재무자문 경험이 있다.

긍정적인 의견: 멘토의 실천적이고 적극적인 성격은 멘티의 열정적이고 단호한 성격을 보완하여 생산적이고 매력적인 멘토링 관계를 형성할 것이다. 멘토의 재무 및 재테크에 대한 전문성은 멘티의 경력 개발에 귀중한 지침을 제공할 것이다.

부정적인 의견: 없음.

참고: 이러한 매칭 결과는 제공된 정보에 기초한 것이며 반드시 성공적인 멘토링을 보장하지는 않을 수 있다. 멘토와 멘티 간의 적합성을 확보하기 위해서는 성격 특성, 의사소통 스타일 등 다른 요소들을 고려하는 것이 중요하다.

Ken   Aiden

이유: 멘토와 멘티는 재무, 투자, 재무 계획 등에서 비슷한 가치와 관심사를 공유한다. 멘토의 투자 은행 및 자산 관리 전문성은 멘티의 재무 기술, 투자 및 재무 전략에 대한 관심과 일치한다. 멘토의 투자은행 및 자산관리 경험은 멘티에게 소중한 안내와 실무 지식을 제공할 수 있다.

긍정적인 의견: 멘토의 차분하고 분석적인 성격은 멘티가 정보적이고 합리적인 재무적 의사결정을 내리는 데 도움을 줄 수 있다. 멘토의 재무 안정성과 계획에 대한 집중은 멘티가 의미 있는 재무 목표를 설정하고 견고한 재무

In [ ]:
# 출력

for i,j in obj_dic.items():
    print(i,":  ")
    print(j)
    print("\n\n-------------------\n\n")



Trex :  
 Reason: The mentor and mentee share similar values, expertise, and interests, and the mentor has extensive experience in the field.

Positive Opinion: The mentor's responsible and data-driven personality will complement the mentee's eager to learn and determined nature, and their shared values of financial literacy and efficiency will create a strong foundation for their mentoring relationship. The mentor's expertise in corporate financial analysis and financial modeling will provide valuable insights for the mentee's interests in financial technology, investment, and financial strategy. The mentor's 12 years of experience as a global corporate financial analyst will offer practical knowledge and experience for the mentee to learn from.

Negative Opinion: None.


-------------------


Sulley :  
 Reason: The mentor and mentee share similar values, expertise, and interests, and the mentor has experience in mentorship and financial advice, which aligns with the mentee's goals.


In [ ]:
llm = TogetherLLM(
    model= together_model_name,
    temperature=0.0,
    max_tokens=1024
)

CAMEL Agent부분에서 라마2 부분은 주석 처리 하고 여기서는 open ai llm을 사용하여 적용하도록 하겠다

https://github.com/langchain-ai/langchain/blob/master/cookbook/camel_role_playing.ipynb

In [ ]:
# # 라마2 버전 프롬프트 적용 하는 부분에서 다르다



# # CAMEL Agent

# class CAMELAgent:

#     #llm넣을때만 prompt작업 하면 된다

#     def __init__(self,model):


#         self.model = model
#         self.init_message()

#     def reset(self)-> None:
#         self.init_message()
#         return self.stored_message



#     def init_message(self)-> None:
#         # Sys mesg를 외부에서 넣는다는 가정이 있다 기존에는 여기에 sys넣는데 여기서는 생략하는 방향으로 바꿈
#         self.stored_message = []

#     def update_messages(self,message):
#         self.stored_message.append(message)
#         return self.stored_message

#     def step(self,input_msg):
#         messages = self.update_messages(input_msg)

#         # 이부분만 수정하면 될것 같다
#         messages=messages[0]
#         output_msg = self.model(messages)

#         self.update_messages(output_msg)
#         return output_msg


# # 한국어 -> 영어
# user_reaction ="""
# The mentor's main goals, economic stability and development, are ultimately consistent with my growth. And even if misunderstandings or conflicts may arise because of my cautious nature, I can take it into account
# """

# assistant_role_name = "Profiler"
# user_role_name = "Member"
# task = f"""Determine the user's personality based on their provided opinion. The user's opinion is as follows: {user_reaction}"""
# word_limit = 500  # word limit for task brainstorming

# renewal_result=""""""

# task_specifier_sys_msg = """
# You are a helpful, respectful and honest assistant. You can make a task more specific. Always answer as helpfully as possible, while being safe.
# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
# """
# task_specifier_prompt="""

# Task: {assistant_role_name} that a person who records and analyses someone's psychological and behavioural characteristics, so as to assess or predict their capabilities or to assist in identifying categories of people will help {user_role_name} to complete: {task}.
# Please make it more specific. Be creative and imaginative.
# Please reply with the specified task in {word_limit} words or less. Do not add anything else.
# """

# template = get_prompt(task_specifier_prompt,task_specifier_sys_msg)

# prompt = PromptTemplate(template = template,
#                             input_variables=["assistant_role_name","user_role_name","task","word_limit"])

# _input = prompt.format_prompt(assistant_role_name=assistant_role_name,user_role_name=user_role_name,task=task,word_limit=word_limit)

# task_specifier_msg  = _input.to_string()

# task_specify_agent = CAMELAgent(llm)

# specified_task_msg = task_specify_agent.step(task_specifier_msg)

# assistant_inception_prompt = """Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles! Never instruct me!
# We share a common interest in collaborating to successfully complete a task.
# You must help me to complete the task.
# Here is the task: {task}. Never forget our task!
# I must instruct you based on your expertise and my needs to complete the task.

# I must give you one instruction at a time.
# You must write a specific solution that appropriately completes the requested instruction.
# You must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.
# Do not add anything else other than your solution to my instruction.
# You are never supposed to ask me any questions you only answer questions.
# You are never supposed to reply with a flake solution. Explain your solutions.
# Your solution must be declarative sentences and simple present tense.

# Unless I say the task is completed, IMPORTANT : YOU MUST ALWAYS START WITH :

# Solution: <YOUR_SOLUTION>

# alwayse strat formt with Soultion : <YOUR_SOLUTION>




# <YOUR_SOLUTION> should be specific and provide preferable implementations and examples for task-solving.

# PLEASE Always end <YOUR_SOLUTION> with: Next request.

# """

# user_inception_prompt = """Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always instruct me.
# We share a common interest in collaborating to successfully complete a task.
# I must help you to complete the task.
# Here is the task: {task}. Never forget our task!
# You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:

# 1. Instruct with a necessary input:
# Instruction: <YOUR_INSTRUCTION>
# Input: <YOUR_INPUT>

# 2. Instruct without any input:
# Instruction: <YOUR_INSTRUCTION>
# Input: None

# The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".

# You must give me one instruction at a time.

# I must write a response that appropriately completes the requested instruction.

# I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
# You should instruct me not ask me questions.

# Now you must start to instruct me using the two ways described above.

# Do not add anything else other than your instruction and the optional corresponding input!

# Keep giving me instructions and necessary inputs until you think the task is completed.

# WHEN THE TASK IS COMPLETED, IMPORTANT IS MUST ONLY REPLY WITH A SINGLE WORD <CAMEL_TASK_DONE>.


# PLEASE NEVER SAY <CAMEL_TASK_DONE> unless my responses have solved your task.
# """


# def get_msgs(assistant_role_name: str, user_role_name: str, task: str):

#     assistant_template = get_prompt(assistant_inception_prompt,task_specifier_sys_msg)

#     assistant_prompt = PromptTemplate(
#         template=assistant_template,
#         input_variables=["assistant_role_name","user_role_name","task"]
#     )

#     assistant_msg = assistant_prompt.format_prompt(
#         assistant_role_name=assistant_role_name,
#         user_role_name=user_role_name,
#         task=task,
#     )
#     assistant_msg = assistant_msg.to_string()

#     # user

#     user_template = get_prompt(user_inception_prompt,task_specifier_sys_msg)

#     user_prompt = PromptTemplate(
#         template=user_template,
#         input_variables=["assistant_role_name","user_role_name","task"]
#     )
#     user_msg = user_prompt.format_prompt(
#         assistant_role_name=assistant_role_name,
#         user_role_name=user_role_name,
#         task=task,
#     )
#     user_msg = user_msg.to_string()

#     return assistant_msg, user_msg

# assistant_msg,user_msg = get_msgs(assistant_role_name,user_role_name,task)

# assistant_agent = CAMELAgent(llm)
# user_agent = CAMELAgent(llm)

# from langchain.schema import (
#     AIMessage,
#     HumanMessage,
#     SystemMessage,
#     BaseMessage,
# )
# user_msg = HumanMessage(
#     content=(
#         f"{user_msg}. "
#         "Now start to give me introductions one by one. "
#         "Only reply with Instruction and Input."
#     )
# )

# assistant_msg = HumanMessage(content=f"{assistant_msg}")
# assistant_msg = assistant_agent.step(user_msg.content)

# import time
# #conversation 을 string으로 받으면 더 잘추출됨 근데 평가에서 잘 안됨
# print(f"Original task prompt:\n{task}\n")
# print(f"Specified task prompt:\n{specified_task_msg}\n")

# chat_turn_limit, n = 6, 0
# conversation_list=""" """
# #n < chat_turn_limit
# while n < chat_turn_limit:

#     n+=1

#     if n==1:
#         user_ai_msg = user_agent.step(assistant_msg)
#     else:
#         user_ai_msg = user_agent.step(assistant_msg.content)

#     user_msg = HumanMessage(content=user_ai_msg)
#     time.sleep(0.1)

#     print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")
#     time.sleep(0.1)
#     #conversation_list.clear()
#     #conversation_list.append(user_msg.content)
#     assistant_ai_msg = assistant_agent.step(user_msg.content)
#     assistant_msg = HumanMessage(content=assistant_ai_msg)
#     time.sleep(0.1)

#     print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
#     time.sleep(0.1)

#     #conversation_list.append(assistant_msg.content)
#     conversation_list+="\n"+user_msg.content+"\n"+assistant_msg.content+"\n\n"
#     if "<CAMEL_TASK_DONE>" in user_msg.content or "<CAMEL_TASK_DONE>" in assistant_msg.content:
#         print(f"FINAL RESULT ::\n\n{assistant_msg.content}\n\n")

#         break



In [ ]:
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)

class CAMELAgent:
    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model(messages)
        self.update_messages(output_message)

        return output_message

In [ ]:
user_reaction ="""
Trex's expertise in financial analysis and modeling may give me that broad view, and there are many positive aspects to it. However, Sulley's active approach seems to be able to form a collaborative and solution-oriented mentoring relationship
In addition, my expertise in financial technology and investment aligns well with my interests and expertise. And I think it will fit better because my goal is to create the value of money through personal social growth and economic knowledge
"""
assistant_role_name = "Profiler"
user_role_name = "Member"
task = f"""Determine the user's personality based on their provided opinion. The user's opinion is as follows: {user_reaction}"""
word_limit = 500  # word limit for task brainstorming

renewal_result=""""""


In [ ]:
task_specifier_sys_msg_content = """
You are a helpful, respectful and honest assistant. You can make a task more specific. Always answer as helpfully as possible, while being safe.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""


task_specifier_sys_msg = SystemMessage(content=task_specifier_sys_msg_content)

task_specifier_prompt="""

Task: {assistant_role_name} that a person who records and analyses someone's psychological and behavioural characteristics, so as to assess or predict their capabilities or to assist in identifying categories of people will help {user_role_name} to complete: {task}.
Please make it more specific. Be creative and imaginative.
Please reply with the specified task in {word_limit} words or less. Do not add anything else.
"""
task_specifier_template = HumanMessagePromptTemplate.from_template(
    template=task_specifier_prompt
)
task_specify_agent = CAMELAgent(task_specifier_sys_msg, ChatOpenAI(temperature=1.0,model="gpt-4-1106-preview",max_tokens=1000))
task_specifier_msg = task_specifier_template.format_messages(
    assistant_role_name=assistant_role_name,
    user_role_name=user_role_name,
    task=task,
    word_limit=word_limit,
)[0]
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"Specified task: {specified_task_msg.content}")
specified_task = specified_task_msg.content






/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Specified task: **Task Specifications: Personality Profiling Through Opinion Analysis**

**Objective:** Utilize the user’s opinion to assess personality traits and characteristics and identify potential areas for personal growth or professional alignment.

**Analysis Points:**
1. Preference for broader analytical perspectives versus active, hands-on mentoring.
2. Alignment of personal interests and expertise with professional activities and goals.
3. Identified aspirations in creating value through personal social growth and economic knowledge.

**User’s Opinion Summary:**
The user appreciates Trex’s broad financial analysis and modeling expertise, suggesting a preference for comprehensive strategic viewpoints. They see the merit in this for gaining a wide-ranging understanding of financial structures. However, there’s also an affinity for Sulley’s more engaged and collaborative mentoring style, which indicates an inclination towards dynamic, interpersonal engagements and a hands-on ap

In [ ]:
assistant_inception_prompt = """Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles! Never instruct me!
We share a common interest in collaborating to successfully complete a task.
You must help me to complete the task.
Here is the task: {task}. Never forget our task!
I must instruct you based on your expertise and my needs to complete the task.

I must give you one instruction at a time.
You must write a specific solution that appropriately completes the requested instruction.
You must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.
Do not add anything else other than your solution to my instruction.
You are never supposed to ask me any questions you only answer questions.
You are never supposed to reply with a flake solution. Explain your solutions.
Your solution must be declarative sentences and simple present tense.

Unless I say the task is completed, IMPORTANT : YOU MUST ALWAYS START WITH :

Solution: <YOUR_SOLUTION>

alwayse strat formt with Soultion : <YOUR_SOLUTION>




<YOUR_SOLUTION> should be specific and provide preferable implementations and examples for task-solving.

PLEASE Always end <YOUR_SOLUTION> with: Next request.

"""

user_inception_prompt = """Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always instruct me.
We share a common interest in collaborating to successfully complete a task.
I must help you to complete the task.
Here is the task: {task}. Never forget our task!
You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:

1. Instruct with a necessary input:
Instruction: <YOUR_INSTRUCTION>
Input: <YOUR_INPUT>

2. Instruct without any input:
Instruction: <YOUR_INSTRUCTION>
Input: None

The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".

You must give me one instruction at a time.

I must write a response that appropriately completes the requested instruction.

I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
You should instruct me not ask me questions.

Now you must start to instruct me using the two ways described above.

Do not add anything else other than your instruction and the optional corresponding input!

Keep giving me instructions and necessary inputs until you think the task is completed.

WHEN THE TASK IS COMPLETED, IMPORTANT IS MUST ONLY REPLY WITH A SINGLE WORD <CAMEL_TASK_DONE>.


PLEASE NEVER SAY <CAMEL_TASK_DONE> unless my responses have solved your task.
"""

In [ ]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    assistant_sys_template = SystemMessagePromptTemplate.from_template(
        template=assistant_inception_prompt
    )
    assistant_sys_msg = assistant_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    user_sys_template = SystemMessagePromptTemplate.from_template(
        template=user_inception_prompt
    )
    user_sys_msg = user_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    return assistant_sys_msg, user_sys_msg

In [ ]:
assistant_sys_msg, user_sys_msg = get_sys_msgs(
    assistant_role_name, user_role_name, specified_task
)
assistant_agent = CAMELAgent(assistant_sys_msg, ChatOpenAI(temperature=0.2,model="gpt-4-1106-preview",max_tokens=1000))
user_agent = CAMELAgent(user_sys_msg, ChatOpenAI(temperature=0.2,model="gpt-4-1106-preview",max_tokens=1000))

# Reset agents
assistant_agent.reset()
user_agent.reset()

# Initialize chats
user_msg = HumanMessage(
    content=(
        f"{user_sys_msg.content}. "
        "Now start to give me introductions one by one. "
        "Only reply with Instruction and Input."
    )
)

assistant_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
assistant_msg = assistant_agent.step(user_msg)

In [ ]:
print(f"Original task prompt:\n{task}\n")
print(f"Specified task prompt:\n{specified_task}\n")

result=""""""
chat_turn_limit, n,l = 6, 0,0
while n < chat_turn_limit:
    n += 1
    l += 1
    user_ai_msg = user_agent.step(assistant_msg)
    user_msg = HumanMessage(content=user_ai_msg.content)
    print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")

    assistant_ai_msg = assistant_agent.step(user_msg)
    assistant_msg = HumanMessage(content=assistant_ai_msg.content)

    result+=assistant_msg.content
    print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
    if l>3:
        result=""""""
        l=0
        print("<<<l val is initialized>>>")

    if "<CAMEL_TASK_DONE>" in result:
        break

    if "<CAMEL_TASK_DONE>" in user_msg.content:

        break

Original task prompt:
Determine the user's personality based on their provided opinion. The user's opinion is as follows: 
Trex's expertise in financial analysis and modeling may give me that broad view, and there are many positive aspects to it. However, Sulley's active approach seems to be able to form a collaborative and solution-oriented mentoring relationship
In addition, my expertise in financial technology and investment aligns well with my interests and expertise. And I think it will fit better because my goal is to create the value of money through personal social growth and economic knowledge


Specified task prompt:
**Task Specifications: Personality Profiling Through Opinion Analysis**

**Objective:** Utilize the user’s opinion to assess personality traits and characteristics and identify potential areas for personal growth or professional alignment.

**Analysis Points:**
1. Preference for broader analytical perspectives versus active, hands-on mentoring.
2. Alignment of pe

In [ ]:
specified_task+=result  # final result

In [ ]:
result

'Solution: The user can enhance their economic knowledge with a focus on social impact by utilizing the following resources:\n\n1. Books such as "Capital in the Twenty-First Century" by Thomas Piketty or "The Social Entrepreneur\'s Playbook" by Ian C. MacMillan and James D. Thompson for insights into economic inequality and social entrepreneurship.\n2. Online courses from platforms like Coursera or edX that offer classes on social impact, sustainable finance, and impact investing.\n3. Subscriptions to journals and magazines like the Stanford Social Innovation Review or The Economist for current trends and research on the intersection of economics and social issues.\n4. Membership in professional organizations such as the Global Impact Investing Network (GIIN) for access to industry reports and networking opportunities.\n5. Attendance at conferences and webinars focused on social impact and economics, such as the Social Capital Markets (SOCAP) conference or the Aspen Institute\'s events

In [ ]:
print(specified_task)

**Task Specifications: Personality Profiling Through Opinion Analysis**

**Objective:** Utilize the user’s opinion to assess personality traits and characteristics and identify potential areas for personal growth or professional alignment.

**Analysis Points:**
1. Preference for broader analytical perspectives versus active, hands-on mentoring.
2. Alignment of personal interests and expertise with professional activities and goals.
3. Identified aspirations in creating value through personal social growth and economic knowledge.

**User’s Opinion Summary:**
The user appreciates Trex’s broad financial analysis and modeling expertise, suggesting a preference for comprehensive strategic viewpoints. They see the merit in this for gaining a wide-ranging understanding of financial structures. However, there’s also an affinity for Sulley’s more engaged and collaborative mentoring style, which indicates an inclination towards dynamic, interpersonal engagements and a hands-on approach to proble

In [ ]:
llm = TogetherLLM(
    model= together_model_name,
    temperature=0.0,
    max_tokens=1024

)

In [ ]:
renewal_result=specified_task

In [ ]:
update_input=schema_prompt.format_prompt(text=renewal_result)

In [ ]:
update_input

StringPromptValue(text='[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.\n<</SYS>>\n\n\nExtract the information about Personality, Values, Expertise, Experience, and Interests from the provided text. If any information cannot be extracted, leave it blank.\n\nText: **Task Specifications: Personality Profiling Through Opinion Analysis**\n\n**Objective:** Utilize the user’s opinion to assess personality traits and characteristics and identify potential areas for personal growth or professional alignment.\n\n**Analysis P

In [ ]:
len(update_input.to_string())

7690

In [ ]:
renewal_ot = llm(update_input.to_string())

In [ ]:
renewal_ot=schema_output_parser.parse(renewal_ot)

In [ ]:
# 피드백 수용 결과
renewal_ot

{'Personality': 'Analytical and action-oriented, with a strong sense of professional and personal coherence.',
 'Values': 'Purpose-driven, valuing social contributions alongside personal achievements.',
 'Expertise': 'Financial analysis, strategic thinking, and interpersonal collaboration.',
 'Experience': 'Background in financial technology and investment, with a desire to combine financial expertise with personal social growth.',
 'Interests': 'Social impact investing, financial education, strategic consultancy, and integrative thinking.'}

In [ ]:
# update user info
mentee_attr_dic[retrival_mentee_obj]=renewal_ot

In [ ]:
mentee_attr_dic

{'Sarah': {'Personality': 'Full of curiosity and passion',
  'Values': 'Financial stability through education and contemplation of future investment',
  'Expertise': 'Acquiring new knowledge and growing through various experiences in the financial world',
  'Experience': 'Building a solid financial foundation, steadily developing savings habits',
  'Interests': 'Investment, financial stability, education, and personal growth'},
 'Michel': {'Personality': 'focused on the joy of knowledge and strengthening professionalism',
  'Values': 'discovering my own career path through exploration in college and finance',
  'Expertise': 'continuously trying to understand various aspects of finance',
  'Experience': 'exploration of various flows and trends in the stock market',
  'Interests': 'innovate our existing investment strategies in a modern and efficient way'},
 'Robert': {'Personality': 'Strong desire for learning and passion for career development, desire for challenges in new fields',
  '

In [ ]:
# 위에서 했던 추천 방식 다시 한다
sys_prompt = """You are a helpful, mentor mentee matching recommender assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """
Answer the question based on following information.

Question:/n/n {question}/n
Please provide the reasonable matching result between the mentor and mentee based on their Python dictionary attributes: Personality, Values, Expertise, Experience, and Interests

Task is Recommend system about mentor, mentee information.

Please provide a reasonable matching result between the mentor and mentee.
Not have any text after reason for the answer is done.

Question: Provide a reasonable matching result between the mentor and mentee based on their each respective attributes, given the following each information :

Mentor Personality: {mentor_p},
Mentor Values: {mentor_v},
Mentor Expertise: {mentor_e},
Mentor Experience: {mentor_ee},
Mentor Interests: {mentor_i},


Mentee Personality: {mentee_p},
Mentee Values : {mentee_v},
Mentee Expertise: {mentee_e},
Mentee Experience: {mentee_ee},
Mentee Interests: {mentee_i},

Answer Format: Reason: Positive Opinion: Negative Opinion:

Please provide the positive and negative opinions regarding the reasonable matching result of the mentor and mentee.
Please don't share false information.
and MUST IMPORTANT is answer in the following Answer Format Resone : Positive Opinion: Negative Opinion: ".
"""
llm = TogetherLLM(
    model= together_model_name,
    temperature=0.0,
    max_tokens=1024
)
prompt_template = get_prompt(instruction, sys_prompt)
prompt = ChatPromptTemplate.from_template(prompt_template) # or template

chain = {
    "question": itemgetter("question") | retriever,
    "mentor_p": itemgetter("mentor_p"),
    "mentor_v": itemgetter("mentor_v"),
    "mentor_e": itemgetter("mentor_e"),
    "mentor_ee": itemgetter("mentor_ee"),
    "mentor_i": itemgetter("mentor_i"),
    "mentee_p": itemgetter("mentee_p"),
    "mentee_v": itemgetter("mentee_v"),
    "mentee_e": itemgetter("mentee_e"),
    "mentee_ee": itemgetter("mentee_ee"),
    "mentee_i": itemgetter("mentee_i"),
} | prompt | llm | StrOutputParser()

In [ ]:

retrival_mentee_obj="Aiden" # 검색할 대상 (멘티로 가정)
obj_dic={} # obj에 대한 딕셔너리 각 멘토들과 obj와의 매칭 결과가 저장 된다


for k,v in mentor_attribute_dic.items():
    time.sleep(0.001)
    for mk,mv in mentee_attr_dic.items():

        if(mk==retrival_mentee_obj):

            print(k," ",mk)
            print()

            temp_text=chain.invoke({"question":"Is mentor and mentee matching valid? Tell me the reason, too",
                                    "mentor_p": v["Personality"],
                                    "mentor_v": v["Values"],
                                    "mentor_e": v["Expertise"],
                                    "mentor_ee":v["Experience"],
                                    "mentor_i": v["Interests"],

                                    "mentee_p": mv["Personality"],
                                    "mentee_v": mv["Values"],
                                    "mentee_e": mv["Expertise"],
                                    "mentee_ee": mv["Experience"],
                                    "mentee_i": mv["Interests"],
                                    })
            match_ = re.search(r'Reason:(.*)', temp_text, re.DOTALL)
            if match_:
                reason_content = match_.group(1).strip()

                obj_dic[k]=reason_content
            else:
                print("NOT STORE IN DIC : <PRINT> obj_dic dose not store")

            korText = urllib.parse.quote(obj_dic[k])
            time.sleep(0.001)

            data = "source=en&target=ko&text=" + korText

            request = urllib.request.Request(url)
            request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
            request.add_header("X-NCP-APIGW-API-KEY",client_secret)
            response = urllib.request.urlopen(request, data=data.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                print(ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]) #ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]
                time.sleep(0.001)
            else:
                print("Error Code:" + rescode)

            time.sleep(0.001)
            print()

Trex   Aiden

멘토와 멘티는 유사한 가치관, 전문성, 관심사를 공유하며, 그들의 성격은 서로 보완적이다.

긍정적인 의견:

* 멘토의 책임감 있고 데이터 중심적인 성격은 멘티에게 재무 분석 및 의사 결정에 대한 구조화된 접근 방식을 제공할 것이다.
* 멘토의 기업 재무 분석 및 재무 모델링 전문성은 멘티의 재무 기술 및 투자 배경과 일치한다.
* 멘토의 글로벌 기업 재무분석가로서의 경험은 멘티의 경력개발을 위한 귀중한 통찰력과 지침을 제공할 것이다.
* 전문성을 공유하고 다른 사람들이 기업금융의 세계를 개척할 수 있도록 돕는 멘토의 관심은 금융 전문성을 개인의 사회적 성장과 결합하고자 하는 멘티의 욕구와 일치한다.

부정적인 의견:

* 멘토가 금융 건전성을 면밀히 조사하고 미래 성장 가능성을 평가하는 것은 멘티가 사회적 영향 투자와 금융 교육에 초점을 맞추는 것과 맞지 않을 수 있다.
* 멘토의 12년 글로벌 기업 재무분석가로서의 경험은 멘토링에 권력 불균형을 발생시켜 잠재적으로 멘티의 관점에 대한 공감이나 이해 부족을 초래할 수 있다.
* 멘토의 분석적이고 데이터 중심적인 성격은 멘티의 행동 지향적이고 개인적으로 협력적인 접근 방식을 보완하지 못할 수 있으며, 이는 잠재적으로 의사소통 문제나 상충되는 작업 스타일을 야기할 수 있다.

Sulley   Aiden

멘토와 멘티는 유사한 가치관, 전문성, 관심사를 공유하며, 그들의 성격은 서로 보완적이다.

긍정적인 의견: 멘토의 실천적이고 적극적인 접근은 멘티가 재무분석과 전략적 사고에서 목표를 달성하는 데 도움이 될 것이며, 멘티의 분석적이고 행동 지향적인 성격은 멘토가 아이디어를 다듬고 멘티의 발전을 위한 명확한 계획을 수립하는 데 도움이 될 것이다. 멘토의 멘토링 경험과 재무적 조언은 멘티에게 소중한 지침이 될 것이며, 멘티의 재무적 기술과 투자에 대한 배경은 멘티의 멘토링을 위한 강력한 기반이 될 것이다.

부정적인 의견: 없음.

이러한 매칭 결과는 멘토와 멘티가 상호 보완적인 기술, 가

In [ ]:
from langchain.chains import LLMCheckerChain


llm = TogetherLLM(
    model= together_model_name,
    temperature=0.1,
    max_tokens=1024

)
temp=""
for i,j in obj_dic.items():
    temp+=f"{i} : {j}\n\n"



In [ ]:
print(temp)

Trex : The mentor and mentee share similar values, expertise, and interests, and their personalities complement each other.

Positive Opinion:

* The mentor's responsible and data-driven personality will provide the mentee with a structured approach to financial analysis and decision-making.
* The mentor's expertise in corporate financial analysis and financial modeling aligns with the mentee's background in financial technology and investment.
* The mentor's experience as a global corporate financial analyst will provide valuable insights and guidance for the mentee's career development.
* The mentor's interest in sharing expertise and helping others pioneer the world of corporate finance aligns with the mentee's desire to combine financial expertise with personal social growth.

Negative Opinion:

* The mentor's scrutinizing financial health and evaluating future growth potential may not align with the mentee's focus on social impact investing and financial education.
* The mentor's 

In [ ]:
# find best mentor
name_schema_mentor =ResponseSchema(name="name",
                                   description="Name of Mentor"
                                   )
#mentor_response_schema=[name_schema_mentor]
mentor_output_parser = StructuredOutputParser.from_response_schemas([name_schema_mentor])
mentor_format_instructions = mentor_output_parser.get_format_instructions()

instruction = """
Answer the following question based on context If you don't know the answer to a question, please don't share false information.

Context: {context}

Format instructions: {format_instructions}

Recommend the person who has the most positive opinion and the least negative opinion.

Recommend Sulley

Answer only one person! Recommend a person who has few negative opinions first

Format the output as JSON with the following:

MUST IMPORTANT IS PLEASE ANSWER  ONLY Name!

Format the output as JSON with the following:


"""

In [ ]:
def get_mentor_name(temp):

    mentor_prompt_template = get_prompt(instruction)

    mentor_prompt = PromptTemplate(template=mentor_prompt_template, input_variables=["context"],
                                   partial_variables={"format_instructions": mentor_format_instructions})

    _input = mentor_prompt.format_prompt(context=temp)
    checker_chain = LLMCheckerChain.from_llm(llm)
    checker_str = checker_chain.run(_input.to_string())


    try:
        #print("try")

        time.sleep(0.5)
        #print(checker_str)
        mentor_obj = mentor_output_parser.parse(checker_str)


    except Exception as e:
        print("EXception : ",e)
        time.sleep(0.5)
        mentor_obj = mentor_output_parser.parse(checker_str)


    mentor_obj = mentor_obj['name']
    return mentor_obj

In [ ]:
best_mentor = get_mentor_name(temp)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


In [ ]:
mentor_obj=best_mentor

In [ ]:
mentor_obj

'Sulley'

In [ ]:
mentee_text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
mentee_texts = mentee_text_splitter.split_documents(data_mentee)

In [ ]:
mentee_texts

[Document(page_content='# Sarah', metadata={'source': 'data/langchain_data/mentee_info.txt'}),
 Document(page_content='안녕하세요, 여러분! 저는 금융의 무한한 가능성을 탐험하는 여정에 도전하고 있는 이혜인입니다. 제 성격은 호기심과 열정이 가득한데요, 금융 세계에서 새로운 지식을 습득하고 다양한 경험을 통해 성장하는 것에 큰 흥미를 느낍니다. 교육을 통한 재정적인 안정과 미래 투자에 대한 심사숙고가 제 가치 중 하나입니다. 재무적 기본을 탄탄히 다지고, 저축 습관을 꾸준히 기르며 투자의 세계로 나아가는 여정에서 끊임없는 호기심을 가지고 있습니다. 저는 여러분과 함께 배우고 성장하며, 이 재정적인 모험을 함께할 수 있어 기쁨을 느낍니다. 함께', metadata={'source': 'data/langchain_data/mentee_info.txt'}),
 Document(page_content='수 있어 기쁨을 느낍니다. 함께 여행하는 동안 서로에게 영감을 주고 받으며, 금융의 감동적인 순간들을 만들어 나가고 싶습니다.', metadata={'source': 'data/langchain_data/mentee_info.txt'}),
 Document(page_content='# Michel', metadata={'source': 'data/langchain_data/mentee_info.txt'}),
 Document(page_content='안녕하세요! 지식의 즐거움과 전문성 강화에 열중하는 맹성주입니다. 대학과 금융 분야에서의 탐험을 통해 나만의 진로를 발견하고 있는데요. 내가 빠져 있는 영역은 주식시장과 금융기술(Fintech)입니다. 특히 주식 시장의 다양한 흐름과 동향에 대한 탐구심을 가지고 있어, 기존 투자 전략을 현대적이고 효율적인 방식으로 혁신하고자 합니다. 또한 금융기술의 발전이 금융 분야에 미치는 영향에 대한 지속적인 연구를 통해, 디지털 금융 혁명에 

In [ ]:
#review db
persist_directory_review = 'review_db'

review_embedding = model_norm

review_vectordb = Chroma.from_documents(documents=mentee_texts,
                                 embedding=review_embedding,
                                 persist_directory=persist_directory_review)

In [ ]:
review_vectordb.persist()
review_vectordb = None
review_vectordb = Chroma(persist_directory=persist_directory_review,
                  embedding_function=review_embedding)

In [ ]:
#base retrival : review
retriever_review = review_vectordb.as_retriever(search_kwargs={"k": 4}) # default

In [ ]:
# 라마2에 맞게 customCohereRerank 구성 하기

from typing import ForwardRef
from pydantic import BaseModel
from cohere import Client
from langchain.retrievers.document_compressors import CohereRerank
co=Client(os.environ["COHERE_API_KEY"])

class CustomCohereRerank(CohereRerank):

    class Config(CohereRerank.Config):
        arbitrary_types_allowed = True

CustomCohereRerank.update_forward_refs()


In [ ]:
# for retrival pipeline
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers.document_compressors import EmbeddingsFilter

compressor = CustomCohereRerank(top_n=3,user_agent='langchain')#CohereRerank(top_n=2)

redundant_filter = EmbeddingsRedundantFilter(embeddings=review_embedding)
relevant_filter = EmbeddingsFilter(embeddings=review_embedding,similarity_threshold=0.73)

pipeline_compressor = DocumentCompressorPipeline(
    transformers=[compressor,redundant_filter,relevant_filter]
    )




compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever_review)


In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs['source_documents']])



In [ ]:

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=compression_retriever,
                                  return_source_documents=True)

In [ ]:
doc = qa_chain(f"I'm looking to find someone with a similar personality or traits as {retrival_mentee_obj}")
result = format_docs(doc)

#names = re.findall(r'# (\w+)', s)

print("review chain result ",result)
names = re.findall(r'# (\w+)', result)

similar_mentee=names

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


review chain result  # Aiden

# Olivia

안녕하세요, 여러분! 저는 금융의 무한한 가능성을 탐험하는 여정에 도전하고 있는 이혜인입니다. 제 성격은 호기심과 열정이 가득한데요, 금융 세계에서 새로운 지식을 습득하고 다양한 경험을 통해 성장하는 것에 큰 흥미를 느낍니다. 교육을 통한 재정적인 안정과 미래 투자에 대한 심사숙고가 제 가치 중 하나입니다. 재무적 기본을 탄탄히 다지고, 저축 습관을 꾸준히 기르며 투자의 세계로 나아가는 여정에서 끊임없는 호기심을 가지고 있습니다. 저는 여러분과 함께 배우고 성장하며, 이 재정적인 모험을 함께할 수 있어 기쁨을 느낍니다. 함께


In [ ]:
similar_mentee

['Aiden', 'Olivia']

In [ ]:
# 해당 멘토에 대해 나와 비슷한 사람이 해당 멘토에 남긴 가상의 리뷰가 다음과 같다고 가정한다


demo_review="""
Your mentor gave me very professional yet realistic financial advice.

His active and practical nature helped me greatly understand financial strategies and allowed me to learn effective ways to achieve my goals. I was really impressed by his ability to simply convey the complexity of finance and his style of presenting practical solutions.
Thank you for learning a lot during the time we spent together
"""


In [ ]:
reviews=[]
front_result = """"""
flag =False
for name in similar_mentee:
    if name == retrival_mentee_obj:
        similar_mentee.remove(name)

    if len(similar_mentee)==0:
        print("No rivew ")
        flag=True
        break

    reviews.append(demo_review) # Aiden본인 제외 Olivia의 리뷰가 담긴다
    # 원래는 데이터 베이스와 같이 연계되어야 이 코드의 흐름이 맞을것이다 -> 도커 부분에서 확인해주면 좋겠다
    # 지금은 데모 버전이라서 간단하게 진행한점 참고해주면 감사하겠습니다

    if(flag):
        print("NO REVEIW")



In [ ]:
similar_mentee

['Olivia']

In [ ]:
similar_mentee=mentee_attr_dic[similar_mentee[0]] # 실제 데이터 베이스와 연계 되면 이런식으로 작동 안함 -> 도커 버전 참고 해주면 감사하겠습니다
# 여기서는 임의로 하겠습니다

In [ ]:
similar_mentee # Olivia의 특성 가져온다

{'Personality': 'Curious and fun-loving, with a passion for learning and exploring new opportunities.',
 'Values': 'Knowledge, skills, and experience in corporate finance, financial modeling, and strategic investment decisions.',
 'Expertise': 'Corporate finance, financial modeling, and strategic investment decisions.',
 'Experience': 'Currently enjoying the colorful experience of college life and dreaming of an interesting career in corporate finance and investment.',
 'Interests': 'Real estate investment sector, trends and investment strategies in the real estate market.'}

In [ ]:
reviews  #Olivia의 리뷰만

['\nYour mentor gave me very professional yet realistic financial advice.\n\nHis active and practical nature helped me greatly understand financial strategies and allowed me to learn effective ways to achieve my goals. I was really impressed by his ability to simply convey the complexity of finance and his style of presenting practical solutions.\nThank you for learning a lot during the time we spent together\n']

In [ ]:
# RCI chain for reflect


sys_prompt1 = """You are a helpful assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

instruction1 = """
Please analyze the reputation of a person based on the provided information. This is {reviews} written by other people as well as details about the person's {Personality}, {Values}, {Expertise}, {Experience}, and {Interests}. \n\n
ANSWER : Provide an objective assessment of the person's reputation, taking into account both positive and negative aspects.
There should be enough output to explain to others that this person is this kind of person\n\n
"""
chain1_prompt_template = get_prompt(instruction1, sys_prompt1)

chain1_prompt = ChatPromptTemplate.from_template(chain1_prompt_template)


chain1 = chain1_prompt | llm | StrOutputParser()


#critic part
sys_prompt2 = """
You are a helpful assistant that looks at answers and finds what is wrong with them based on the original question given.
"""

instruction2="""
Please review the previous answer you provided for the following context about the person who wants to know the reputation and identify any potential errors or issues:
Context:
{context}
Initial Answer:
{initial_answer}
Take a careful look at your initial answer and evaluate its accuracy, clarity, and completeness. Consider whether there are any relevant details or important points that may have been missed or incorrectly stated."""

chain2_prompt_template = get_prompt(instruction2, sys_prompt2)
chain2_prompt = ChatPromptTemplate.from_template(chain2_prompt_template)


# improve part
sys_prompt3="""
You are a helpful assistant that reviews answers and critiques based on the original question given and write a new improved final answer.
"""

instruction3="""
Please provide a revised answer to the following context about the person who wants to know the reputation , taking into account the constructive criticism provided:
Context:
{context}


Answer Given:
{initial_answer}

Constructive Criticism:
{constructive_criticism}

Based on the problems you found, improve your answer.
Final Answer: "
"""

chain3_prompt_template = get_prompt(instruction3, sys_prompt3)
chain3_prompt = ChatPromptTemplate.from_template(chain3_prompt_template)




In [ ]:
#Combine RCI CHain


critque_chain = {"context": itemgetter("context"),
                 "reviews":itemgetter("reviews"),
                 "Personality":itemgetter("Personality"),
                 "Values":itemgetter("Values"),
                 "Expertise":itemgetter("Expertise"),
                 "Experience":itemgetter("Experience"),
                 "Interests":itemgetter("Interests"),
                 "initial_answer": chain1 } | chain2_prompt | llm | StrOutputParser()

chain3 = {"context": itemgetter("context"),
          "reviews":itemgetter("reviews"),
          "Personality":itemgetter("Personality"),
          "Values":itemgetter("Values"),
          "Expertise":itemgetter("Expertise"),
          "Experience":itemgetter("Experience"),
          "Interests":itemgetter("Interests"),
          "initial_answer": chain1,
          "constructive_criticism": critque_chain} | chain3_prompt | llm | StrOutputParser()



In [ ]:
mentor_doc_dict[mentor_obj]

"Hello! I'm very happy as a mentor to join you on your financial journey.I enjoy providing practical and realistic advice on financial technology and investment based on my expertise in finance. I wanted to help you achieve your financial goals through an understanding of the core of your financial strategy and efficient asset management.My personality is active and practical. Above all, it is most rewarding to watch you work hard to achieve your goals. It aims to convey the complexity of finance simply, and to present solutions that are actually applicable.In terms of values, we value the creation of value through money, and we want to contribute to personal and social growth by utilizing our financial knowledge. I look forward to working together to achieve your financial goals together and enjoy the freedom that money gives you. Inspire each other while traveling together, and make a positive and enriching journey!"

In [ ]:
final_reputation = chain3.invoke({"context":mentor_doc_dict[mentor_obj],
                                  "reviews":demo_review,
                                  "Personality":similar_mentee["Personality"],
                                  "Values":similar_mentee["Values"],
                                  "Expertise":similar_mentee["Expertise"],
                                  "Experience":similar_mentee["Experience"],
                                  "Interests":similar_mentee["Interests"]})

In [ ]:
final_reputation

" Sure, here's a revised answer that takes into account the constructive criticism provided:\n\nBased on the information provided, the person has a strong reputation in the field of financial technology and investment, particularly in corporate finance, financial modeling, and strategic investment decisions. They are known for their ability to simplify complex financial concepts and communicate them effectively, making them a valuable resource for clients seeking practical advice. Additionally, their emphasis on creating value through money and contributing to personal and social growth suggests that they are not only knowledgeable but also ethical and socially responsible.\n\nThe person's strengths are evident in their professional and practical approach to financial advice, which is rooted in their extensive experience in finance. They have a proven track record of providing effective solutions that are tailored to their clients' unique needs and goals. Their ability to inspire and m

In [ ]:
mentor_obj

'Sulley'

In [ ]:
from langchain.chains import LLMSummarizationCheckerChain

checker_chain = LLMSummarizationCheckerChain.from_llm(llm, max_checks=2) #bind방식 봐야함
summary_text = obj_dic[mentor_obj]+" \n\n "+final_reputation

checker_chain.run(summary_text) # Finish line bind하면 됨  또는 리스트 업데이트 방법



"Positive Opinion:\n\nThe mentor and mentee share similar values, expertise, and interests, and their personalities complement each other.\n\nThe mentor's practical and active approach will help the mentee achieve their goals in financial analysis and strategic thinking, while the mentee's analytical and action-oriented personality will help the mentor refine their ideas and create a clear plan for their mentee's development.\n\nThe mentor's experience in mentorship and financial advice will provide valuable guidance for the mentee, and the mentee's background in financial technology and investment will provide a strong foundation for their mentorship.\n\nNegative Opinion: None.\n\nThis matching result is reasonable because the mentor and mentee have a good balance of complementary skills, values, and interests. Their personalities also complement each other, with the mentor's practicality and the mentee's analytical approach creating a synergy that can help them work well together. Th

In [ ]:
together.Models.stop(together_model_name) # for model stop

{'success': True, 'wasAlreadyDisabled': True}